In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
import selenium
from pandas import DataFrame
import json

In [13]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [50]:
browser = init_browser()
url = "https://www.fragrantica.com/search/"
browser.visit(url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/jenniferdean/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [51]:
html = browser.html
soup = bs(html, "html.parser")

In [5]:
list_of_perfume_dicts = []

In [42]:
for i in range(1923,2021):
    #try:
        ####### CHANGE THE YEAR SETTING #######
        browser.find_by_css('input[type="number"]')[1].fill(i)
        browser.find_by_css('input[type="number"]')[0].fill(i)
        time.sleep(5)

        ####### CLICK "SHOW MORE RESULTS" #######
        ####### with this coded out, we only get 40 results per year ######
        for j in range(0,100):
            try:
                browser.find_by_css('button[class="button"]').click()
            except:
                break
            
        html = browser.html
        soup = bs(html, "html.parser")

        ####### GET THE LENGTH OF THE RESULTS #######
        block1 = soup.find("span", class_="grid-x grid-margin-x grid-margin-y small-up-3 medium-up-2 large-up-4 perfumes-row text-center")
        perfume_name_list = []
        
        for perf_text in block1.find_all("a", href=True):
            perf_name = perf_text.get_text()
            perfume_name_list.append(perf_name.replace('\n', '').strip(" "))

        ####### START ITERATING THROUGH THE PERFUMES OF THAT YEAR #######    
        for k in range(len(perfume_name_list)):
            html = browser.html
            soup = bs(html, "html.parser")

            browser.find_by_css('span[class="link-span"]')[k].click()

            time.sleep(5)

            html = browser.html
            soup = bs(html, "html.parser")

            time.sleep(5)

            ####### GET DATA FROM THE PURFUME PAGE #######
            perfume_name = soup.find_all("div", class_="cell small-12")[3].find_all("b")[0].get_text()
            perfume_comp = soup.find_all("div", class_="cell small-12")[3].find_all("b")[1].get_text()

            perfume_image = soup.find_all("div", class_="cell small-12")[1].find("img")["src"]

            for_gender = soup.find("small").get_text()
            
            try:
                rating = float(soup.find("p", class_="info-note").find_all("span")[0].get_text())
                number_votes = int(soup.find("p", class_="info-note").find_all("span")[2].get_text().replace(',', ''))
            except:
                rating = "NA"
                number_votes = "NA"
                print(f"{perfume_name} does not have a ranking")
                
            try:
                description = soup.find_all("div", class_="cell small-12")[3].get_text()
            except:
                description = "NA"
                print(f"{perfume_name} does not have a description")

            ####### MAIN ACCORDS DICTIONARY #######
            
            try:
                main_accords = soup.find_all("div", class_="cell accord-box")
                accords_dict = {}
                for m in range(len(main_accords)):
                    accord_name = main_accords[m].get_text()
                    accord_value = float(main_accords[m].find("div", class_="accord-bar")["style"].rsplit("width: ")[1].strip("%;"))
                    accords_dict[accord_name] = accord_value
            except:
                accords_dict = {}
                print(f"{perfume_name} does not have accords")

            ####### FRAGRANCE NOTES #######        
            notes = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; text-align: center; flex-flow: row wrap; align-items: flex-end; padding: 0.5rem;"})

            if len(notes) == 3:
                number = 2
                top_notes_list = []
                middle_notes_list = []
                base_notes_list = []

                for n in range(len(notes[0].find_all("span", class_="link-span"))):
                    top_notes_list.append(notes[0].find_all("div")[number].get_text())
                    number += 3

                number = 2
                for p in range(len(notes[1].find_all("span", class_="link-span"))):
                    middle_notes_list.append(notes[1].find_all("div")[number].get_text())
                    number += 3

                number = 2
                for q in range(len(notes[2].find_all("span", class_="link-span"))):
                    base_notes_list.append(notes[2].find_all("div")[number].get_text())
                    number += 3
            elif len(notes) == 2:
                number = 2
                top_notes_list = []
                middle_notes_list = []
                base_notes_list = []

                for r in range(len(notes[0].find_all("span", class_="link-span"))):
                    top_notes_list.append(notes[0].find_all("div")[number].get_text())
                    number += 3

                number = 2
                for s in range(len(notes[1].find_all("span", class_="link-span"))):
                    middle_notes_list.append(notes[1].find_all("div")[number].get_text())
                    number += 3
            elif len(notes) == 1:
                number = 2
                top_notes_list = []
                middle_notes_list = []
                base_notes_list = []

                for v in range(len(notes[0].find_all("span", class_="link-span"))):
                    middle_notes_list.append(notes[0].find_all("div")[number].get_text())
                    number += 3
            else:
                top_notes_list = []
                middle_notes_list = []
                base_notes_list = []

            ####### VOTING DATA & INFORMATION #######
            voting = soup.find_all("div", class_="cell small-1 medium-1 large-1")

            ####### Longevity #######
            long_v_weak = int(voting[0].get_text())
            long_weak = int(voting[1].get_text())
            long_moderate = int(voting[2].get_text())
            long_long_last = int(voting[3].get_text())
            long_eternal = int(voting[4].get_text())

            ####### Sillage #######
            sill_intimate = int(voting[5].get_text())
            sill_moderate = int(voting[6].get_text())
            sill_strong = int(voting[7].get_text())
            sill_enormus = int(voting[8].get_text())

            ####### Gender #######
            gender_female = int(voting[9].get_text())
            gender_more_fem = int(voting[10].get_text())
            gender_unisex = int(voting[11].get_text())
            gender_more_male = int(voting[12].get_text())
            gender_male = int(voting[13].get_text())

            ####### Price Value #######
            value_w_over = int(voting[14].get_text())
            value_over = int(voting[15].get_text())
            value_ok = int(voting[16].get_text())
            value_good = int(voting[17].get_text())
            value_great = int(voting[18].get_text())

            ####### CREATING THE DICTIONARY OF DATA #######
            perfume_dict = {"name": perfume_name,
                            "company": perfume_comp,
                            "image": perfume_image,
                            "for_gender": for_gender,
                            "rating": rating,
                            "number_votes": number_votes,
                            "main accords": accords_dict,
                            "description": description,
                            "top notes": top_notes_list,
                            "middle notes": middle_notes_list,
                            "base notes": base_notes_list,
                            "longevity":   {"very weak": long_v_weak,
                                            "weak": long_weak,
                                            "moderate": long_moderate,
                                            "long lasting": long_long_last,
                                            "eternal": long_eternal},
                            "sillage":     {"intimate": sill_intimate,
                                            "moderate": sill_moderate,
                                            "strong": sill_strong,
                                            "enormous": sill_enormus},
                            "gender_vote": {"female": gender_female,
                                            "more female": gender_more_fem,
                                            "unisex": gender_unisex,
                                            "more male": gender_more_male,
                                            "male": gender_male},
                            "price value": {"way overpriced": value_w_over,
                                            "overpriced": value_over,
                                            "ok": value_ok,
                                            "good value": value_good,
                                            "great value": value_great}
                           }
            list_of_perfume_dicts.append(perfume_dict)

            time.sleep(2)

            browser.back()
            
            time.sleep(2)
        
#     except:
#         print(f"Error with {i} year - skipping to next year")
#         print(f"-------------------------------------------")

Violetera does not have a ranking
Le Jardin de Ma Soeur does not have a ranking
Le Beau Soir does not have a ranking
Ptah does not have a ranking
Valée des Rois does not have a ranking
Misti does not have a ranking
Flo does not have a ranking


IndexError: list index out of range

In [43]:
len(list_of_perfume_dicts)

75

In [30]:
list_of_perfume_dicts

[{'name': 'Knize Ten',
  'company': 'Knize',
  'image': 'https://fimgs.net/mdimg/perfume/375x500.7913.jpg',
  'for_gender': 'for men',
  'rating': 4.22,
  'number_votes': 902,
  'main accords': {'leather': 100.0,
   'woody': 81.6293,
   'animalic': 66.6779,
   'warm spicy': 65.8188,
   'aromatic': 64.7002,
   'smoky': 63.2817,
   'fresh spicy': 56.4778,
   'powdery': 56.201,
   'citrus': 56.0165,
   'musky': 54.5461},
  'description': "Knize Ten by Knize is a Leather fragrance for men. Knize Ten was launched during the 1920's. Knize Ten was created by Vincent Roubert and Francois Coty. Top notes are Geranium, Rosemary, Bergamot, Orange, Petitgrain and Amalfi Lemon; middle notes are Carnation, Patchouli, Sandalwood, Orris, tincture of rose, Virginia Cedar, Cinnamon and Orange Blossom; base notes are Leather, Castoreum, oak moss, Amber, Musk and Vanilla. Knize Ten, a classic and a unique scent, is considered to be one of the best leather compositions in the history of perfume. It was lau

In [52]:
with open ("perfume_data_1920_1924.json", "w") as f:
    json.dump(list_of_perfume_dicts, f)

In [48]:
####################################################
####### USE THIS FOR A SINGULAR PERFUME PAGE #######
############### NOT THE SEARCH PAGE ###############
###################################################

html = browser.html
soup = bs(html, "html.parser")

time.sleep(1)

####### PULL FROM ORIGINAL NAME LIST #######
perfume_name = soup.find_all("div", class_="cell small-12")[3].find_all("b")[0].get_text()
perfume_comp = soup.find_all("div", class_="cell small-12")[3].find_all("b")[1].get_text()

perfume_image = soup.find_all("div", class_="cell small-12")[1].find("img")["src"]

for_gender = soup.find("small").get_text()

try:
    rating = float(soup.find("p", class_="info-note").find_all("span")[0].get_text())
    number_votes = int(soup.find("p", class_="info-note").find_all("span")[2].get_text().replace(',', ''))
except:
    rating = "NA"
    number_votes = "NA"
    print(f"{perfume_name} does not have a ranking")

try:
    description = soup.find_all("div", class_="cell small-12")[3].get_text()
except:
    description = "NA"
    print(f"{perfume_name} does not have a description")

####### MAIN ACCORDS DICTIONARY #######

try:
    main_accords = soup.find_all("div", class_="cell accord-box")
    accords_dict = {}
    for m in range(len(main_accords)):
        accord_name = main_accords[m].get_text()
        accord_value = float(main_accords[m].find("div", class_="accord-bar")["style"].rsplit("width: ")[1].strip("%;"))
        accords_dict[accord_name] = accord_value
except:
    accords_dict = {}
    print(f"{perfume_name} does not have accords")

####### FRAGRANCE NOTES #######        
notes = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; text-align: center; flex-flow: row wrap; align-items: flex-end; padding: 0.5rem;"})

if len(notes) == 3:
    number = 2
    top_notes_list = []
    middle_notes_list = []
    base_notes_list = []

    for n in range(len(notes[0].find_all("span", class_="link-span"))):
        top_notes_list.append(notes[0].find_all("div")[number].get_text())
        number += 3

    number = 2
    for p in range(len(notes[1].find_all("span", class_="link-span"))):
        middle_notes_list.append(notes[1].find_all("div")[number].get_text())
        number += 3

    number = 2
    for q in range(len(notes[2].find_all("span", class_="link-span"))):
        base_notes_list.append(notes[2].find_all("div")[number].get_text())
        number += 3
elif len(notes) == 2:
    number = 2
    top_notes_list = []
    middle_notes_list = []
    base_notes_list = []

    for r in range(len(notes[0].find_all("span", class_="link-span"))):
        top_notes_list.append(notes[0].find_all("div")[number].get_text())
        number += 3

    number = 2
    for s in range(len(notes[1].find_all("span", class_="link-span"))):
        middle_notes_list.append(notes[1].find_all("div")[number].get_text())
        number += 3
elif len(notes) == 1:
    number = 2
    top_notes_list = []
    middle_notes_list = []
    base_notes_list = []

    for v in range(len(notes[0].find_all("span", class_="link-span"))):
        middle_notes_list.append(notes[0].find_all("div")[number].get_text())
        number += 3
else:
    top_notes_list = []
    middle_notes_list = []
    base_notes_list = []

####### VOTING DATA & INFORMATION #######
voting = soup.find_all("div", class_="cell small-1 medium-1 large-1")

####### Longevity #######
long_v_weak = int(voting[0].get_text())
long_weak = int(voting[1].get_text())
long_moderate = int(voting[2].get_text())
long_long_last = int(voting[3].get_text())
long_eternal = int(voting[4].get_text())

####### Sillage #######
sill_intimate = int(voting[5].get_text())
sill_moderate = int(voting[6].get_text())
sill_strong = int(voting[7].get_text())
sill_enormus = int(voting[8].get_text())

####### Gender #######
gender_female = int(voting[9].get_text())
gender_more_fem = int(voting[10].get_text())
gender_unisex = int(voting[11].get_text())
gender_more_male = int(voting[12].get_text())
gender_male = int(voting[13].get_text())

####### Price Value #######
value_w_over = int(voting[14].get_text())
value_over = int(voting[15].get_text())
value_ok = int(voting[16].get_text())
value_good = int(voting[17].get_text())
value_great = int(voting[18].get_text())

####### CREATING THE DICTIONARY OF DATA #######
perfume_dict = {"name": perfume_name,
                "company": perfume_comp,
                "image": perfume_image,
                "for_gender": for_gender,
                "rating": rating,
                "number_votes": number_votes,
                "main accords": accords_dict,
                "description": description,
                "top notes": top_notes_list,
                "middle notes": middle_notes_list,
                "base notes": base_notes_list,
                "longevity":   {"very weak": long_v_weak,
                                "weak": long_weak,
                                "moderate": long_moderate,
                                "long lasting": long_long_last,
                                "eternal": long_eternal},
                "sillage":     {"intimate": sill_intimate,
                                "moderate": sill_moderate,
                                "strong": sill_strong,
                                "enormous": sill_enormus},
                "gender_vote": {"female": gender_female,
                                "more female": gender_more_fem,
                                "unisex": gender_unisex,
                                "more male": gender_more_male,
                                "male": gender_male},
                "price value": {"way overpriced": value_w_over,
                                "overpriced": value_over,
                                "ok": value_ok,
                                "good value": value_good,
                                "great value": value_great}
               }
list_of_perfume_dicts.append(perfume_dict)

time.sleep(2)

browser.back()

Isabey Gardenia does not have a ranking


In [49]:
list_of_perfume_dicts

[{'name': 'Knize Ten',
  'company': 'Knize',
  'image': 'https://fimgs.net/mdimg/perfume/375x500.7913.jpg',
  'for_gender': 'for men',
  'rating': 4.22,
  'number_votes': 902,
  'main accords': {'leather': 100.0,
   'woody': 81.6293,
   'animalic': 66.6779,
   'warm spicy': 65.8188,
   'aromatic': 64.7002,
   'smoky': 63.2817,
   'fresh spicy': 56.4778,
   'powdery': 56.201,
   'citrus': 56.0165,
   'musky': 54.5461},
  'description': "Knize Ten by Knize is a Leather fragrance for men. Knize Ten was launched during the 1920's. Knize Ten was created by Vincent Roubert and Francois Coty. Top notes are Geranium, Rosemary, Bergamot, Orange, Petitgrain and Amalfi Lemon; middle notes are Carnation, Patchouli, Sandalwood, Orris, tincture of rose, Virginia Cedar, Cinnamon and Orange Blossom; base notes are Leather, Castoreum, oak moss, Amber, Musk and Vanilla. Knize Ten, a classic and a unique scent, is considered to be one of the best leather compositions in the history of perfume. It was lau